In [1]:
import ee
import geemap.foliumap as geemap

# ee.Authenticate()
ee.Initialize()

In [2]:
def get_landsat_data(start_date, end_date, geometry):
    landsat_data = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').select('SR_B[1-7]').filterDate(start_date, end_date).filterBounds(geometry)
    return landsat_data

fresno = ee.Geometry.Point([-120.0, 36.0])

landsat_may = get_landsat_data('2019-05-01', '2019-06-01', fresno).first()
landsat_jun = get_landsat_data('2019-06-01', '2019-07-01', fresno).first()
landsat_jul = get_landsat_data('2019-07-01', '2019-08-01', fresno).first()
landsat_aug = get_landsat_data('2019-08-01', '2019-09-01', fresno).first()
landsat_sep = get_landsat_data('2019-09-01', '2019-10-01', fresno).first()
landsat_oct = get_landsat_data('2019-10-01', '2019-11-01', fresno).first()

In [3]:
vis_params = {'min': 0, 'max': 30000, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}
modis_vis = {'min': 0.0, 'max': 100.0, 'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']}

m = geemap.Map(center=(36, -120), zoom=8)
m.addLayer(landsat_may, vis_params, 'May')
m.addLayer(landsat_jun, vis_params, 'June')
m.addLayer(landsat_jul, vis_params, 'July')
m.addLayer(landsat_aug, vis_params, 'August')
m.addLayer(landsat_sep, vis_params, 'September')
m.addLayer(landsat_oct, vis_params, 'October')
# m.addLayer(cdl.eq(61), {}, 'CDL')
m.addLayerControl()
m

In [4]:
cdl_orig = ee.ImageCollection('USDA/NASS/CDL').filterDate('2019-01-01', '2019-12-31').select('cropland').first().clip(landsat_sep.geometry())
cultivated = ee.ImageCollection('USDA/NASS/CDL').filterDate('2019-01-01', '2019-12-31').select('cultivated').first().clip(landsat_sep.geometry())
cdl_cultivated = cdl_orig.mask(cultivated.eq(2))
cdl = cdl_cultivated.reproject(crs=landsat_sep.projection(), scale=30)

In [5]:
area = ee.Image.pixelArea().addBands(cdl)
 
areas = area.reduceRegion(reducer=ee.Reducer.sum().group(groupField=1, groupName='class'), 
                          geometry=cdl.geometry(), scale=30, maxPixels=1e10)

class_areas = ee.List(areas.get('groups'))

def get_area(item):
    area_dict = ee.Dictionary(item)
    class_number = ee.Number(area_dict.get('class')).format()
    area = ee.Number(area_dict.get('sum')).divide(1e6).round()
    return ee.List([class_number, area])

class_area_lists = class_areas.map(get_area)
 
result = ee.Dictionary(class_area_lists.flatten())
sorted_keys = result.keys().sort(result.values())

n_classes = 6
class_keys = sorted_keys.getInfo()[-n_classes:]
class_keys

# tot = 0
# for each in sorted_keys.getInfo():
#     tot = tot + result.getInfo()[each]
#     print(each, result.getInfo()[each])
# print(tot)

['36', '2', '204', '69', '75', '61']

In [6]:
main_crops = cdl.expression("b('cropland') == 75 || b('cropland') == 69 || b('cropland') == 204 || b('cropland') == 2 || b('cropland') == 36")
cdl = cdl.mask(main_crops)

In [7]:
def get_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  #NIR, R
    return ndvi

In [8]:
def get_lai(start_date, end_date):
    lai = ee.ImageCollection('MODIS/061/MCD15A3H').filterDate(start_date, end_date).select('Lai').first().clip(landsat_sep.geometry())
    lai = lai.mask(main_crops)
    lai = lai.resample('bicubic').reproject(crs=landsat_sep.projection(), scale=30)
    return lai

In [9]:
def get_final_image(image, bands):
    data = image.addBands(bands)
    return data

In [10]:
def get_training_data(image, start_date, end_date):
    lai = get_lai(start_date, end_date)
    landsat_image = image.mask(main_crops)
    ndvi = get_ndvi(landsat_image)
    bands = [ndvi, lai]
    data = get_final_image(landsat_image, bands)
    return data

In [11]:
data_may = get_training_data(landsat_may, '2019-05-01', '2019-06-01')
data_jul = get_training_data(landsat_jul, '2019-07-01', '2019-08-01')
data_aug = get_training_data(landsat_aug, '2019-08-01', '2019-09-01')
data_sep = get_training_data(landsat_sep, '2019-09-01', '2019-10-01')
data_oct = get_training_data(landsat_oct, '2019-10-01', '2019-11-01')

In [12]:
points = cdl.stratifiedSample(region=cdl.geometry(), scale=30, numPoints=1500, seed=0, geometries=True)

label = 'cropland'
bands = data_sep.bandNames().getInfo()


def classification(image):
    sample = image.select(bands).sampleRegions(collection=points, properties=[label], scale=30)
    sample = sample.randomColumn()
    split = 0.70

    training = sample.filter(ee.Filter.lt('random', split))
    validation = sample.filter(ee.Filter.gte('random', split))
    
    return training, validation

t_may, v_may = classification(data_may)
t_jul, v_jul = classification(data_jul)
t_aug, v_aug = classification(data_aug)
t_sep, v_sep = classification(data_sep)
t_oct, v_oct = classification(data_oct)

training = t_may.merge(t_jul).merge(t_aug).merge(t_sep).merge(t_oct)
validation = v_may.merge(v_jul).merge(v_aug).merge(v_sep).merge(v_oct)

In [13]:
classifier = ee.Classifier.smileRandomForest(200).train(training, label, bands)

train_accuracy = classifier.confusionMatrix()
train_accuracy.accuracy().getInfo()

0.9989940447448897

In [14]:
validated = validation.classify(classifier)

test_accuracy = validated.errorMatrix('cropland', 'classification')
test_accuracy.accuracy().getInfo()

0.6887009992313605

In [15]:
points = cdl.stratifiedSample(region=cdl.geometry(), scale=30, numPoints=20000, seed=0, geometries=True)

label = 'cropland'
bands = data_sep.bandNames().getInfo()


def classification(image):
    sample = image.select(bands).sampleRegions(collection=points, properties=[label], scale=30)   
    return sample

t_may = classification(data_may)
t_jul = classification(data_jul)
t_aug = classification(data_aug)
t_sep = classification(data_sep)
t_oct = classification(data_oct)

training = t_may.merge(t_jul).merge(t_aug).merge(t_sep).merge(t_oct)

In [16]:
task = ee.batch.Export.table.toDrive(collection=training,
                                     description='training_500k',
                                     fileFormat='CSV')
task.start()

In [ ]:
task.status()